In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
537,tt0240462,70000000,176104344,Dr. Dolittle 2,Eddie Murphy|Kristen Wilson|Raven-SymonÃ©|Kyla...,Steve Carr,The doctor is in again.,"Dr. John Dolittle the beloved doctor is back, ...",87,Comedy|Family|Romance|Fantasy,Twentieth Century Fox Film Corporation,6/22/2001,4.9,2001
1328,tt0397535,85000000,162242962,Memoirs of a Geisha,Zhang Ziyi|Gong Li|Youki Kudoh|Tsai Chin|Suzuk...,Rob Marshall,My world is as forbidden as it is fragile; wit...,A sweeping romantic epic set in Japan in the y...,145,Drama|History|Romance,DreamWorks SKG|Spyglass Entertainment|Columbia...,12/6/2005,7.1,2005
871,tt0120912,140000000,441818803,Men in Black II,Tommy Lee Jones|Will Smith|Rip Torn|Lara Flynn...,Barry Sonnenfeld,Same Planet. New Scum.,"Kay and Jay reunite to provide our best, last ...",88,Action|Adventure|Comedy|Science Fiction,Columbia Pictures|Amblin Entertainment|Columbi...,7/3/2002,5.9,2002
211,tt3322940,6500000,255273813,Annabelle,Annabelle Wallis|Alfre Woodard|Eric Ladin|Tony...,John R. Leonetti,"Before the Conjuring, there was Annabelle.",John Form has found the perfect gift for his e...,99,Horror,New Line Cinema|The Safran Company|Evergreen M...,10/2/2014,5.6,2014
1160,tt0289992,50000000,38955598,The Life of David Gale,Kevin Spacey|Kate Winslet|Laura Linney|Gabriel...,Alan Parker,The crime is clear. The truth is not.,A man against capital punishment is accused of...,130,Drama|Thriller|Crime,Universal Pictures|Intermedia Films|Saturn Films,2/21/2003,7.2,2003


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

### Функции:
def month(release_date):
    """TASK 23 - Выбирает числовое значение месяца из даты формата mm/dd/yyyy, выводит название месяца"""
    month_number = str(release_date).find('/')
    month = int(str(release_date)[:month_number])
    
    calendar = {
        1: "Январь",
        2: "Февраль",
        3: "Март",
        4: "Апрель",
        5: "Май",
        6: "Июнь",
        7: "Июль",
        8: "Август",
        9: "Сентябрь",
        10: "Октябрь",
        11: "Ноябрь",
        12: "Декабрь"
    }
    
    return calendar[month]


def cast_func(cast_list):
  """TASK 27 - Разбивает актеров конкретного фильма по парам, оставляя уникальные пары"""
  cast_together = []
  for name_1 in cast_list:
    for name_2 in cast_list:
      if name_1 == name_2:
        continue
      cast_together += [(name_1, name_2)]
  return cast_together
        

### тут другие ваши предобработки колонок например:

# добавляется новый столбец с прибылью фильма:
data['profit'] = data['revenue'] - data['budget']

# Task 11-12 - учитываем все жанры, потому что есть фильмы, у которых их сразу несколько
new_data_genres = data.copy() # копируем, датафрейм еще раз, чтобы не вносить изменения в копию оригинала
new_data_genres['genres'] = new_data_genres['genres'].str.split('|')
new_data_genres = new_data_genres.explode('genres')

# Task 13 - последовательность действий с режиссерами точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_director = data.copy()
new_data_director['director'] = new_data_director['director'].str.split('|')
new_data_director = new_data_director.explode('director')

# Task 14 - последовательность действий с режиссерами точно такая же, как поступали выше
new_data_director_genre = new_data_director.copy()
new_data_director_genre['genres'] = new_data_director_genre['genres'].str.split('|')
new_data_director_genre = new_data_director_genre.explode('genres')

# Tasks 15-16 - последовательность действий с актерами точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_cast = data.copy()
new_data_cast['cast'] = new_data_cast['cast'].str.split('|')
new_data_cast = new_data_cast.explode('cast')

# Tasks 17 - последовательность действий, как выше
new_data_cast_genre = new_data_cast.copy()
new_data_cast_genre['genres'] = new_data_cast_genre['genres'].str.split('|')
new_data_cast_genre = new_data_cast_genre.explode('genres')

# Task 18,20 - последовательность действий с компаниями точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_company = data.copy()
new_data_company['production_companies'] = new_data_company['production_companies'].str.split('|')
new_data_company = new_data_company.explode('production_companies')

# Task 21-22 - последовательность действий с месяцами точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_month = data.copy()
new_data_month['release_date'] = new_data_month['release_date'].apply(month)
new_data_month = new_data_month.explode('release_date')

# Task 23 - готовим базу одновременно под режиссеров и месяцы по образцам из Tasks 14,21-22 выше
new_data_director_month = new_data_director.copy()
new_data_director_month['release_date'] = new_data_director_month['release_date'].apply(month)
new_data_director_month = new_data_director_month.explode('release_date')

# Task 24 - делаем дополнительный столбец с длиной символов названия каждого фильма, изменяя базу Task 18,20
new_data_title = new_data_company.copy()
new_data_title['title_length'] = new_data_title['original_title'].str.len()

# Task 25 - делаем дополнительный столбец с числом промежутков между словами обзора фильма, изменяя базу Task 18,20
new_data_company_overview = new_data_company.copy()
new_data_company_overview['overview_number'] = new_data_company_overview['overview'].str.count(' ')
new_data_company_overview = new_data_company_overview.explode('overview_number')

# Task 27 - применяем функцию к актерам
new_data_casts = new_data_cast.copy()
new_data_casts['cast'] = new_data_casts['cast'].apply(cast_func)
new_data_casts = new_data_casts.explode('cast')


#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [130]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'

# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [245]:
# тут пишем ваш код для решения данного вопроса:
title_1 = data.loc[data.budget == data.budget.max()].original_title.iloc[0]
print(f'У фильма "{title_1}" самый большой бюджет.')

У фильма "Pirates of the Caribbean: On Stranger Tides" самый большой бюджет.


ВАРИАНТ 2

In [132]:
# можно добавлять разные варианты решения
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000


# 2. Какой из фильмов самый длительный (в минутах)?

In [133]:
# думаю логику работы с этим словарем вы уже поняли, 
# поэтому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [237]:
title_2 = data[data.runtime == data.runtime.max()].original_title.iloc[0]
print(f'"{title_2}" - самый длительный в минутах фильм')

"Gods and Generals" - самый длительный в минутах фильм


In [202]:
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003,-43076064


# 3. Какой из фильмов самый короткий (в минутах)?





In [136]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [137]:
data.loc[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011,-15540000


In [240]:
title_3 = data[data.runtime == data.runtime.min()].original_title.iloc[0]
print(f'"{title_3}" - самый короткий в минутах фильм среди представленных')

"Winnie the Pooh" - самый короткий в минутах фильм среди представленных


# 4. Какова средняя длительность фильмов?


In [139]:
answers['4'] = '2. 110'

In [213]:
time_4 = round(data['runtime'].mean())
print(f'Средняя длительность фильмов составляет {time_4} минут')

Средняя длительность фильмов составляет 110 минут


# 5. Каково медианное значение длительности фильмов? 

In [141]:
answers['5'] = '1. 107'

In [214]:
time_5 = round(data['runtime'].median())
print(f'Медианное значение длительности фильмов - {time_5} минут')

Медианное значение длительности фильмов - 107 минут


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [143]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = '5. Avatar (tt0499549)'

In [10]:
title_6 = data[['imdb_id', 'original_title']][data['profit'] == data['profit']\
                                              .max()].original_title.iloc[0]
id_6 = data[['imdb_id', 'original_title']][data['profit'] == data['profit']\
                                           .max()].imdb_id.iloc[0]
print(f'"{title_6}" - самый прибыльный фильм. Его уникальный номер в базе - "{id_6}".')

"Avatar" - самый прибыльный фильм. Его уникальный номер в базе - "tt0499549".


# 7. Какой фильм самый убыточный? 

In [145]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [11]:
title_7 = data[['imdb_id', 'original_title']][data['profit'] == data['profit'].min()]\
                                                            .original_title.iloc[0]
id_7 = data[['imdb_id', 'original_title']][data['profit'] == data['profit'].min()]\
                                                                    .imdb_id.iloc[0]
print(f'А вот фильм "{title_7}" - самый убыточный. Его уникальный номер в базе - "{id_7}".')

А вот фильм "The Lone Ranger" - самый убыточный. Его уникальный номер в базе - "tt1210819".


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [147]:
answers['8'] = '1. 1478'

In [252]:
num_8 = data[data['revenue'] > data['budget']]['imdb_id'].nunique()
print(f'У {num_8} фильмов объем сборов оказался выше бюджета.')

У 1478 фильмов объем сборов оказался выше бюджета.


In [149]:
data[data['revenue'] > data['budget']].imdb_id.nunique()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [150]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [12]:
inf_9 = data[data.release_year == 2008].sort_values(by='revenue', ascending=False)\
                                            [['original_title', 'imdb_id']].iloc[0][:2]
print(f'В 2008 году самым кассовым фильмом оказался "{inf_9[0]}". \
Его уникальный номер в базе - "{inf_9[1]}".')

В 2008 году самым кассовым фильмом оказался "The Dark Knight". Его уникальный номер в базе - "tt0468569".


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [152]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [6]:
inf_10 = data[data.release_year.isin([2012,2013,2014])].sort_values(by='profit', ascending=True)\
                                                            [['original_title', 'imdb_id']].iloc[0][:2]
print(f'За период с 2012 по 2014 гг. самым убыточным фильмом оказался "{inf_10[0]}". \
Его уникальный номер в базе - "{inf_10[1]}".')

За период с 2012 по 2014 гг. самым убыточным фильмом оказался "The Lone Ranger". Его уникальный номер в базе - "tt1210819".


# 11. Какого жанра фильмов больше всего?

In [154]:
answers['11'] = '3. Drama'

In [263]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genres_11 = new_data_genres.genres.value_counts().index[0]
print(f'Больше всего в базе данных фильмов жанра "{genres_11}".')

Больше всего в базе данных фильмов жанра "Drama".


ВАРИАНТ 2

In [262]:
display(new_data_genres.genres.value_counts().index[0])

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [156]:
answers['12'] = '1. Drama'

In [261]:
genres_12 = new_data_genres[new_data_genres['profit'] > 0].genres.value_counts().index[0]
print(f'Чаще всего фильмы жанра "{genres_12}" становятся прибыльными.')

Чаще всего фильмы жанра "Drama" становятся прибыльными.


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [158]:
answers['13'] = '5. Peter Jackson'

In [313]:
# смотри ПРЕДОБРАБОТКУ датафрейма
director_13 = new_data_director.groupby(['director']).revenue.sum()\
                                .sort_values(ascending=False).index[0]
print(f'Самые большие суммарные кассовые сборы у режиссера {director_13}.')

Самые большие суммарные кассовые сборы у режиссера Peter Jackson.


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [160]:
answers['14'] = '3. Robert Rodriguez'

In [266]:
# смотри ПРЕДОБРАБОТКУ датафрейма
director_14 = new_data_director_genre[new_data_director_genre['genres'] == 'Action']\
        .groupby('director').imdb_id.nunique().sort_values(ascending=False).index[0]
print(f'{director_14} снял больше всего фильмов в стиле Action.')

Robert Rodriguez снял больше всего фильмов в стиле Action.


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [162]:
answers['15'] = '3. Chris Hemsworth'

In [267]:
# смотри ПРЕДОБРАБОТКУ датафрейма
cast_15 = new_data_cast[new_data_cast['release_year'] == 2012]\
        .groupby('cast').revenue.sum().sort_values(ascending=False).index[0]
print(f'Фильмы с {cast_15} принесли самые высокие кассовые сборы в 2012 году.')

Фильмы с Chris Hemsworth принесли самые высокие кассовые сборы в 2012 году.


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [164]:
answers['16'] = '3. Matt Damon'

In [314]:
# смотри ПРЕДОБРАБОТКУ датафрейма
cast_16 = new_data_cast[new_data_cast['budget'] > new_data_cast['budget'].mean()]\
                .groupby('cast').imdb_id.nunique().sort_values(ascending=False).index[0]
print(f'{cast_16} снялся в большем количестве высокобюджетных фильмов.')

Matt Damon снялся в большем количестве высокобюджетных фильмов.


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [169]:
answers['17'] = '2. Action'

In [315]:
# смотри ПРЕДОБРАБОТКУ датафрейма
genre_17 = new_data_cast_genre[new_data_cast_genre['cast'] == 'Nicolas Cage']\
                                                .genres.value_counts().index[0]
print(f'Больше всего  Nicolas Cage снимался в фильмах жанра "{genre_17}".')

Больше всего  Nicolas Cage снимался в фильмах жанра "Action".


# 18. Самый убыточный фильм от Paramount Pictures

In [171]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [271]:
# смотри ПРЕДОБРАБОТКУ датафрейма
title_18 = new_data_company[new_data_company['production_companies'] == 'Paramount Pictures']\
        [['original_title', 'imdb_id', 'profit']].sort_values(by='profit', ascending=True)\
        .head(1).original_title.iloc[0]
print(f'"{title_18}" оказался самым убыточным фильмом от Paramount Pictures.')

"K-19: The Widowmaker" оказался самым убыточным фильмом от Paramount Pictures.


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [173]:
answers['19'] = '5. 2015'

In [272]:
year_19 = data.groupby('release_year').revenue.sum().sort_values(ascending=False).index[0]
print(f'Самым успешным по суммарным кассовым сборам оказался {year_19} год.')

Самым успешным по суммарным кассовым сборам оказался 2015 год.


# 20. Какой самый прибыльный год для студии Warner Bros?

In [175]:
answers['20'] = '1. 2014'

In [273]:
# смотри ПРЕДОБРАБОТКУ датафрейма
year_20 = new_data_company[new_data_company['production_companies']\
        .str.match('Warner Bros', na=False)].groupby('release_year').profit.sum()\
        .sort_values(ascending=False).index[0]
print(f'Самым прибыльным годом для студии Warner Bros оказался {year_20} год.')

Самым прибыльным годом для студии Warner Bros оказался 2014 год.


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [177]:
answers['21'] = '4. Сентябрь'

In [316]:
# смотри ПРЕДОБРАБОТКУ датафрейма
month_21 = new_data_month.groupby('release_date').imdb_id.nunique()\
                                .sort_values(ascending=False).index[0]
print(f'{month_21} - это месяц, в котором за все годы суммарно вышло больше всего фильмов.')

Сентябрь - это месяц, в котором за все годы суммарно вышло больше всего фильмов.


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [179]:
answers['22'] = '2. 450'

In [317]:
# смотри ПРЕДОБРАБОТКУ датафрейма
num_21 = new_data_month[new_data_month['release_date'].isin(['Июнь', 'Июль', 'Август'])]\
                                                                        .imdb_id.nunique()
print(f'Летом суммарно вышло {num_21} фильмов.')

Летом суммарно вышло 450 фильмов.


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [182]:
answers['23'] = '5. Peter Jackson'

In [318]:
# смотри ПРЕДОБРАБОТКУ датафрейма
director_23 = new_data_director_month[new_data_director_month['release_date']\
                        .isin(['Декабрь', 'Январь', 'Февраль'])].groupby('director')\
                        .imdb_id.nunique().sort_values(ascending=False).index[0]
print(f'Для режиссера {director_23} зима оказалась самым продуктивным временем года.')

Для режиссера Peter Jackson зима оказалась самым продуктивным временем года.


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [184]:
answers['24'] = '5. Four By Two Productions'

In [319]:
# смотри ПРЕДОБРАБОТКУ датафрейм
company_24 = new_data_title.groupby('production_companies')['title_length']\
                                    .mean().sort_values(ascending=False).index[0]
print('Студия "{}" дает самые длинные названия своим фильмам по количеству символов.'.format(company_24))

Студия "Four By Two Productions" дает самые длинные названия своим фильмам по количеству символов.


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [186]:
answers['25'] = '3. Midnight Picture Show'

In [320]:
# смотри ПРЕДОБРАБОТКУ датафрейма
company_25 = new_data_company_overview.groupby('production_companies')\
                    .overview_number.mean().sort_values(ascending=False).index[0]
print('У студии "{}" описания фильмов в среднем самые длинные по количеству слов.'.format(company_25))

У студии "Midnight Picture Show" описания фильмов в среднем самые длинные по количеству слов.


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [188]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [321]:
data[data['vote_average'] > data['vote_average'].quantile(.99)]\
            .sort_values(by='vote_average', ascending=False).original_title

599                                   The Dark Knight
9                                          Inside Out
34                                               Room
118                                      Interstellar
125                                The Imitation Game
119                           Guardians of the Galaxy
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [190]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [341]:
# В рабочем файле в Colab у меня отображаются фамилии и имена полностью, здесь - нет
new_data_casts['cast'].value_counts().head()

name1, name2 = new_data_casts.cast.value_counts().index[2]
print(f'Чаще всего в одном фильме снимается пара {name1} & {name2}.')

Чаще всего в одном фильме снимается пара a &  .


# Submission

In [191]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [192]:
# и убедиться что ни чего не пропустил)
len(answers)

27